In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime



In [ ]:
comp = "BBRI"
comp = comp+".JK"
end = datetime.now()
start = datetime(end.year - 5, end.month, end.day)
comp_dat = yf.download(comp, start, end)

In [ ]:
company_list = [comp_dat]
company_name = [comp]

for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name
    
df = pd.concat(company_list, axis=0)
df

In [ ]:
# Create a new dataframe with only the 'Close column 
data = df.filter(['Close'])
# Convert the dataframe to a numpy array
dataset = data.values
# Get the number of rows to train the model on
training_data_len = int(np.ceil( len(dataset) * .80 ))

dataset.shape

In [ ]:
# Normalize the prices between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(dataset)


In [ ]:
# Split the data into training and testing sets
train_size = int(len(scaled_prices) * 0.8)-30
train_data = scaled_prices[:train_size]
test_data = scaled_prices[train_size:]

In [ ]:
test_data

In [ ]:
test_data.shape

In [ ]:
# Step 2: Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length-1):
        X.append(data[i : i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

In [ ]:
seq_length = 30  # Number of previous days' prices to use as input features
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)



In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, batch_size=1, epochs=5)

In [ ]:
X_test.shape

In [ ]:
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error:", mse)

In [ ]:
y_test.shape

In [ ]:
# Get the models predicted price values 
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
predictions.shape

In [ ]:
# Create the testing data set

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse

In [ ]:
valid = data[train_size+seq_length+1:]
valid['Predictions'] = predictions
valid

In [ ]:
# Plot the data
valid['Predictions'] = predictions
# Visualize the data
plt.figure(figsize=(16,6))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(valid[['Close', 'Predictions']])
plt.legend([ 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
# Step 6: Make predictions for the next 7 days
last_sequence = test_data[-seq_length:]  # Last sequence from the testing data

predicted_prices = []

harga_kedepan = []

banyak_prediksi= 7
for _ in range(banyak_prediksi):
    next_price = model.predict(last_sequence.reshape(1, seq_length, 1))
    predicted_prices.append(next_price)
    print(next_price)
    last_sequence = np.append(last_sequence[1:], next_price)
    harga_kedepan.append(next_price)



# Inverse transform the predicted prices to the original scale
predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

# Print the predicted prices for the next 7 days
# print("Predicted Prices for the Next 7 Days:")
# for price in predicted_prices:
#     print(price[0])

print(harga_kedepan)

In [ ]:
predicted_prices[0][0]

In [ ]:
last_sequence

In [ ]:
# Step 6: Make predictions for the next 7 days
last_sequence = test_data[-seq_length:]  # Last sequence from the testing data
predicted_prices = []
harga_lstm = []

banyak_prediksi = 7
for _ in range(banyak_prediksi):
    next_price = model.predict(last_sequence.reshape(1, seq_length, 1))
    predicted_prices.append(next_price)
    last_sequence = np.append(last_sequence[1:], next_price)



    # Inverse transform the predicted prices to the original scale
predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

    # Print the predicted prices for the next 7 days
for price in predicted_prices:
    harga_pred = price[0]
    harga_lstm.append(harga_pred)



In [ ]:
harga_lstm

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime


def prediksi_lstm(df,lama_pred):

    data = df.copy()
    # Create a new dataframe with only the 'Close column 
    data = data.filter(['Close'])
    # Convert the dataframe to a numpy array
    dataset = data.values
    # Get the number of rows to train the model on
    training_data_len = int(np.ceil( len(dataset) * .80 ))-30
    # Normalize the prices between 0 and 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_prices = scaler.fit_transform(dataset)

    # Split the data into training and testing sets
    train_size = int(len(scaled_prices) * 0.8)
    train_data = scaled_prices[:train_size]
    test_data = scaled_prices[train_size:]

    # Step 2: Create sequences
    def create_sequences(data, seq_length):
        X, y = [], []
        for i in range(len(data) - seq_length-1):
            X.append(data[i : i + seq_length])
            y.append(data[i + seq_length])
        return np.array(X), np.array(y)
    
    seq_length = 30  # Number of previous days' prices to use as input features
    X_train, y_train = create_sequences(train_data, seq_length)
    X_test, y_test = create_sequences(test_data, seq_length)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=2)

    # Get the models predicted price values 
    predictions_lstm = []
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    for i in predictions:
        pred_each= i[0]
        predictions_lstm.append(pred_each)


    # Step 6: Make predictions for the next 7 days
    last_sequence = test_data[-seq_length:]  # Last sequence from the testing data
    predicted_prices = []
    harga_lstm = []

    banyak_prediksi = lama_pred
    for _ in range(banyak_prediksi):
        next_price = model.predict(last_sequence.reshape(1, seq_length, 1))
        predicted_prices.append(next_price)
        last_sequence = np.append(last_sequence[1:], next_price)



    # Inverse transform the predicted prices to the original scale
    predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

    # Print the predicted prices for the next 7 days
    for price in predicted_prices:
        harga_pred = price[0]
        harga_lstm.append(harga_pred)

        

    return(harga_lstm,predictions_lstm)





In [2]:
import scrap_data
data_saham = scrap_data.ambil_data("bbri")

data_saham

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,company_name
Date,,,,,,,
2018-06-04,3150.0,3170.0,3100.0,3130.0,2594.125000,211936400,BBRI.JK
2018-06-05,3150.0,3160.0,3090.0,3110.0,2577.549561,135979600,BBRI.JK
2018-06-06,3120.0,3160.0,3090.0,3150.0,2610.701172,99509500,BBRI.JK
2018-06-07,3180.0,3270.0,3160.0,3270.0,2710.156250,183746200,BBRI.JK
2018-06-08,3210.0,3250.0,3110.0,3140.0,2602.413086,262972300,BBRI.JK
...,...,...,...,...,...,...,...
2023-05-25,5500.0,5575.0,5450.0,5475.0,5475.000000,224391900,BBRI.JK
2023-05-26,5425.0,5600.0,5425.0,5600.0,5600.000000,154003100,BBRI.JK
2023-05-29,5600.0,5625.0,5475.0,5550.0,5550.000000,125049100,BBRI.JK


In [3]:
import model_lstm

prediksi = prediksi_lstm(data_saham,7)

prediksi

Epoch 1/2
961/961 [==============================] - 14s 12ms/step - loss: 0.0036
Epoch 2/2
1/1 [==============================] - 0s 25ms/step


([5585.2456, 5611.879, 5646.775, 5688.398, 5735.7837, 5787.943, 5843.831],
 [4160.0107,
  4131.556,
  4140.2036,
  4171.2207,
  4178.034,
  4281.0957,
  4365.663,
  4381.909,
  4357.124,
  4372.7944,
  4381.3545,
  4404.8047,
  4419.1064,
  4423.027,
  4443.7344,
  4446.132,
  4430.6675,
  4417.1035,
  4415.6704,
  4441.1484,
  4405.1943,
  4394.5376,
  4377.9966,
  4357.8696,
  4336.3955,
  4354.4634,
  4358.687,
  4342.34,
  4345.933,
  4347.3604,
  4336.915,
  4328.582,
  4313.4956,
  4309.798,
  4356.211,
  4422.49,
  4496.9316,
  4616.717,
  4668.9062,
  4615.2466,
  4634.608,
  4661.421,
  4645.4116,
  4636.0967,
  4630.866,
  4646.315,
  4595.699,
  4595.094,
  4560.93,
  4572.298,
  4596.817,
  4568.7754,
  4577.3647,
  4584.921,
  4583.7686,
  4558.108,
  4544.514,
  4563.533,
  4646.168,
  4646.9062,
  4609.347,
  4537.0093,
  4449.366,
  4434.9097,
  4431.31,
  4408.8213,
  4350.944,
  4331.601,
  4328.084,
  4339.581,
  4403.2227,
  4474.379,
  4553.8906,
  4646.6934,
  465

In [6]:
prediksi[0]

[5585.2456, 5611.879, 5646.775, 5688.398, 5735.7837, 5787.943, 5843.831]

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime


def prediksi_lstm(df,lama_pred):

    data = df.copy()
    # Create a new dataframe with only the 'Close column 
    data = data.filter(['Close'])
    # Convert the dataframe to a numpy array
    dataset = data.values
    # Get the number of rows to train the model on
    training_data_len = int(np.ceil( len(dataset) * .80 ))
    # Normalize the prices between 0 and 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_prices = scaler.fit_transform(dataset)

    # Split the data into training and testing sets
    train_size = int(len(scaled_prices) * 0.8) - 30
    train_data = scaled_prices[:train_size]
    test_data = scaled_prices[train_size:]

    # Step 2: Create sequences
    def create_sequences(data, seq_length):
        X, y = [], []
        for i in range(len(data) - seq_length-1):
            X.append(data[i : i + seq_length])
            y.append(data[i + seq_length])
        return np.array(X), np.array(y)
    
    seq_length = 30  # Number of previous days' prices to use as input features
    X_train, y_train = create_sequences(train_data, seq_length)
    X_test, y_test = create_sequences(test_data, seq_length)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=2)

    # Get the models predicted price values 
    predictions_lstm = []
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    for i in predictions:
        pred_each= i[0]
        predictions_lstm.append(pred_each)


    # Step 6: Make predictions for the next 7 days
    last_sequence = test_data[-seq_length:]  # Last sequence from the testing data
    predicted_prices = []
    harga_kedepan = []

    banyak_prediksi = lama_pred
    for _ in range(banyak_prediksi):
        next_price = model.predict(last_sequence.reshape(1, seq_length, 1))
        predicted_prices.append(next_price)
        last_sequence = np.append(last_sequence[1:], next_price)



    # Inverse transform the predicted prices to the original scale
    predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

    # Print the predicted prices for the next 7 days
    for price in predicted_prices:
        harga_pred = price[0]
        harga_kedepan.append(harga_pred)

        

    return(harga_kedepan,predictions_lstm)



